In [ ]:
import pandas as pd
import numpy as np
import re
import os
import shutil
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib
from collections import Counter
from collections import defaultdict
import pickle
import datetime

In [ ]:
path = '/path/to/html/and/php/files/downloaded/from/waybackmachine'

## Parse and clean all files in folder

In [ ]:
cnt=0
websites=[]
dfs=[]
for folder in os.listdir(path):
    
    cnt+=1
    print('*'*10)
    print(cnt,folder,'\n',datetime.datetime.now())
    print('*'*10)
    start_count=datetime.datetime.now() ##start timer
    
    # make a list of all files associated with a particular website
    allfiles=[]
    for root, dirc, files in os.walk(os.path.join(path,folder)):
        for file in files:
            allfiles.append(os.path.join(root,file))
                
    print('Total number of files =', len(allfiles))
    
    # parse files and clean resulting text data
    
    keys=[]
    values=[]
    
    for file in allfiles:
        
        try:
            f = open(file, 'rb')
        except Exception as e:
            print('!!! ERROR !!!')
            print(str(e))
            continue
        document= BeautifulSoup(f.read(),'lxml').get_text(strip=False)
        
        if len(document) > 0:

            keys.append(file.split('/')[-2])
            sometxt=document.split(',')    
            sometxt=[item.replace('\n', ' ').replace('\t',' ').replace('\xa0',' ') for item in sometxt]
            moretxt=[]
            for item in sometxt:
                moretxt.extend(item.split(' '))
            moretxt=[item.strip() for item in moretxt]
            words=[item for item in moretxt if 50 > len(item) > 1]
            text=u' '.join(words)
            
            lsst=[]
            lst= re.findall(r'\b[^\d\W]+', str(text))
            lst = [x.lower() for x in lst]
            lst = [x.replace('ä','ae').replace('ö','oe').replace('ü','ue').replace('ß','ss') for x in lst]
            lsst.extend(lst)
            text=u' '.join(lsst)

            values.append(text)
            
    end_count=datetime.datetime.now()     ## stop timer
    tt=end_count-start_count              ## calculate time taken
    print('Time taken was %s \n' % tt)
    
    # create a default dictionary where the keys are the months corresponding to the snapshots and 
    # the values are the text from the snapshots
    
    defdict = defaultdict(list)
    for i, j in zip(keys, values):
        defdict[i].append(j)
        
    # create a dataframe from a regular dictionary where the values from each key of the default dictionary
    # are joined together as one string but seperated by !!!!.
    # Then store this dataframe in a list of dataframes
    
    new_keys=[]
    new_values=[]
    for key in defdict:
        new_keys.append(key)
        new_values.append('!!!!'.join([item for item in defdict.get(key)]))
        dictionary = dict(zip(new_keys, new_values))
    
    df=pd.DataFrame(dictionary,index=[cnt])
    
    websites.append(folder)
    dfs.append(df)
    
    # when the list contains 10 dataframes join them together as 1 dataframe 
    # and export the new dataframe to a pickle file
    
    if cnt%10 == 0:
        
        df_words=pd.concat(dfs, keys = websites, names = ['Website', 'Index'], sort=True)
        outputpath='/path/to/output/folder/parsing_results_%s.p' % str(cnt/10)
        with open(outputpath,'wb') as f:
            pickle.dump(df_words,f)
        print('-§-'*10)
        print('Parsing results %s have been exported' % str(cnt/10) )
        print('-§-'*10,'\n')
        websites=[]
        dfs=[]

# join remaining dataframes into 1 dataframe and export to a pickle file
        
df_words=pd.concat(dfs, keys = websites, names = ['Website', 'Index'], sort=True)
outputpath='/path/to/output/folder/parsing_results_%s.p' % str(cnt/10)
with open(outputpath,'wb') as f:
    pickle.dump(df_words,f)
print('-§-'*10)
print('Parsing results %s have been exported' % str(cnt/10) )
print('-§-'*10,'\n')
    